# Section 8.2: Time Matrix Multiply

Adapted from: "Guide to Fortran 2008 Programming" by Walter S. Brainerd (Springer 2015)

## Program to demonstrate the performance improvement of using matmul over do loops.

```fortran
program time_matrix_multiply

    ! Compare times of the matmul intrinsic cs DO loops

    implicit none
    integer, parameter :: n = 10
    real, dimension (n, n) :: a, b, c1, c2
    character(len=8) :: date
    real :: start_time_1, stop_time_1, start_time_2, stop_time_2
    real :: total_time_1, total_time_2
    integer :: i, j, k
    character(len=*), parameter :: form = "(t2, a, f0.10, a)"

    ! Get date to print on report

    call date_and_time(date = date)

    print *, "Timing report dated: " // date(1:4) &
        // "-" // date(5:6) // "-" // date(7:8)

    call random_seed()
    call random_number(a)
    call random_number(b)
    call cpu_time(start_time_1)
   
    ! Lines below added for effect
    ! The matrices of random values will be printed to the screen
    write (*, "(a)") "Matrix A"
    write (*, "(10f10.3)") (a(i,:), i=1,n)

    print *
    write (*, "(a)") "Matrix B"
    write (*, "(10f10.3)") (b(i,:), i=1,n)

    c1 = 0

    do k = 1, n
        do j = 1, n
            do i = 1, n
                c1(i, j) = c1(i, j) + a(i, k) * b(k, j)
            end do
        end do
    end do

    call cpu_time(stop_time_1)

    total_time_1 = stop_time_1 - start_time_1

    print *

    write (*, "(a)") "Matrix C1 is AxB using loops."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    call cpu_time(start_time_2)
    c2 = matmul(a, b)
    call cpu_time(stop_time_2)

    total_time_2 = stop_time_2 - start_time_2

    print *

    write (*, "(a)") "Matrix C2 is AxB using matmul."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    print *

    write (*, form) "Time of Do loop version is: ", total_time_1, " seconds."
    write (*, form) "Time of matmul version is: ", total_time_2, " seconds."

    print *

    if (any(abs(c1-c2) > 1.0e-4)) then
        write (*,  "(a)") "There are significantly different values between the matrices."
    else
        write (*,  "(a)") "The results are approximately the same."
    end if

    print *
    write (*, "(a, f10.3, a)") "The speedup ratio is: ", total_time_1/total_time_2, "x"

end program time_matrix_multiply
```

The above program is compiled and run using Fortran Package Manager (fpm):

In [1]:
import os
root_dir = os.getcwd()

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_8_2_Time_Matrix_Multiply"

In [3]:
os.chdir(code_dir)

In [4]:
build_status = os.system("fpm build > /dev/null")

fpm run output is piped into sed to suppress the status of the run command and only print the output of the executable.

In [5]:
exec_status = os.system("fpm run | sed 1,1d")

 Timing report dated: 2022-05-30
Matrix A
     0.963     0.733     0.895     0.536     0.000     0.763     0.547     0.409     0.041     0.923
     0.814     0.573     0.240     0.864     0.624     0.226     0.508     0.358     0.021     0.130
     0.953     0.816     0.210     0.331     0.821     0.739     0.755     0.999     0.249     0.794
     0.986     0.021     0.385     0.037     0.975     0.911     0.377     0.290     0.878     0.916
     0.176     0.816     0.657     0.427     0.386     0.530     0.271     0.889     0.865     0.260
     0.299     0.882     0.545     0.767     0.456     0.892     0.401     0.358     0.518     0.449
     0.569     0.414     0.060     0.875     0.475     0.796     0.700     0.846     0.565     0.065
     0.730     0.931     0.562     0.288     0.786     0.800     0.956     0.215     0.038     0.050
     0.609     0.234     0.315     0.062     0.200     0.596     0.802     0.114     0.553     0.026
     0.644     0.201     0.977     0.327     0.46

605     1.677     1.398     1.738     1.180     1.402
     3.428     3.163     3.130     2.452     2.624     3.328     2.418     2.628     2.289     2.591

 Time of Do loop version is: .0002369999 seconds.
 Time of matmul version is: .0000310000 seconds.

The results are approximately the same.

The speedup ratio is:      7.645x
